In [1]:
from transformers import AutoImageProcessor, AutoModelForObjectDetection

c:\source\repos\Models-Inferencing-On-Client-Sandbox\sandboxenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
checkpoint = "hustvl/yolos-tiny"
teacher_processor = AutoImageProcessor.from_pretrained(checkpoint)
teacher_model = AutoModelForObjectDetection.from_pretrained(
    checkpoint,
    ignore_mismatched_sizes=True,
)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [3]:
from datasets import load_dataset, load_from_disk
import os


train_dir = "C:/source/repos/Models-Inferencing-On-Client-Sandbox/datasets/image-face-dataset/train"
test_dir = "C:/source/repos/Models-Inferencing-On-Client-Sandbox/datasets/image-face-dataset/test"
validation_dir = "C:/source/repos/Models-Inferencing-On-Client-Sandbox/datasets/image-face-dataset/validation"

is_data_local: bool = os.path.exists(train_dir) and os.path.exists(test_dir) and os.path.exists(validation_dir)
is_using_subset: bool = True

if is_data_local:
    train_dataset = load_from_disk(train_dir)
    test_dataset = load_from_disk(test_dir)
    validation_dataset = load_from_disk(validation_dir)
else:
    train_dataset = load_dataset("wider_face", split="train[:5%]")
    test_dataset = load_dataset("wider_face", split="test[:5%]")
    validation_dataset = load_dataset("wider_face", split="validation[:5%]")
    train_dataset.save_to_disk(train_dir)
    test_dataset.save_to_disk(test_dir)
    validation_dataset.save_to_disk(validation_dir)

In [4]:
from datasets import DatasetDict

dataset = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'validation': validation_dataset,
})

dataset

DatasetDict({
    train: Dataset({
        features: ['image', 'faces'],
        num_rows: 644
    })
    test: Dataset({
        features: ['image', 'faces'],
        num_rows: 805
    })
    validation: Dataset({
        features: ['image', 'faces'],
        num_rows: 161
    })
})

In [5]:

def add_coco_fields(example, idx):
    bboxes = example['faces']['bbox']
    areas = [bbox[2] * bbox[3] for bbox in bboxes]
    
    example['image_id'] = idx
    example['faces']['area'] = areas
    example['faces']['category'] = [0 for _ in example["faces"]["expression"]]
    
    return example

In [6]:
for split in dataset.keys():
    dataset[split] = dataset[split].map(add_coco_fields, with_indices=True)
    
categories = ["face"]

In [7]:
dataset["train"][0]

{'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=1024x1385>,
 'faces': {'area': [18178.0],
  'bbox': [[449.0, 330.0, 122.0, 149.0]],
  'blur': [0],
  'category': [0],
  'expression': [0],
  'illumination': [0],
  'invalid': [False],
  'occlusion': [0],
  'pose': [0]},
 'image_id': 0}

In [8]:
import albumentations
import numpy as np
import torch

transform = albumentations.Compose(
    [
        albumentations.Resize(480, 480),
        albumentations.HorizontalFlip(p=1.0),
        albumentations.RandomBrightnessContrast(p=1.0),
    ],
    bbox_params=albumentations.BboxParams(format="coco", label_fields=["category"]),
)

In [9]:
def formatted_anns(image_id, category, area, bbox):
    annotations = []
    for i in range(0, len(category)):
        new_ann = {
            "image_id": image_id,
            "category_id": category[i],
            "isCrowd": 1 if len(bbox)>1 else 0,
            "area": area[i],
            "bbox": list(bbox[i]),
        }
        annotations.append(new_ann)

    return annotations

In [10]:
def transform_aug_ann(examples):
    processed_images = []  # This will hold the processed 'pixel_values'
    labels = []  # This will hold the 'labels' dicts for each image
    
    for idx, (image, faces) in enumerate(zip(examples["image"], examples["faces"])):
        # Convert PIL image to numpy array and ensure RGB order
        image_np = np.array(image.convert("RGB"))
        
        # Apply transformations
        transformed = transform(image=image_np, bboxes=faces["bbox"], category=faces["category"])
        
        # Process transformed image with teacher_processor to get pixel_values
        pixel_values = teacher_processor(images=transformed["image"], return_tensors="pt").pixel_values
        
        # Construct labels dict
        image_labels = {
            'size': torch.tensor([480, 480], dtype=torch.long),
            'image_id': torch.tensor([examples["image_id"][idx]], dtype=torch.long),
            'class_labels': torch.tensor(faces["category"], dtype=torch.long),
            'boxes': torch.tensor(transformed["bboxes"]).float(),
            'area': torch.tensor(faces["area"]),
            'iscrowd': torch.tensor([1 if len(faces["bbox"]) > 1 else 0]),
            'orig_size': torch.tensor([image.size[1], image.size[0]])  # Height, Width
        }
        
        # Append the processed pixel_values and labels
        pixel_values = pixel_values.squeeze(0) # Remove extra dim
        processed_images.append(pixel_values)
        labels.append(image_labels)
    
    # Convert lists to tensors for batching
    pixel_values_batch = torch.stack(processed_images)
    # Labels need to be a list of dicts, each dict corresponding to an image
    
    return {"pixel_values": pixel_values_batch, "labels": labels}


In [11]:
dataset["train"] = dataset["train"].with_transform(transform_aug_ann)

In [12]:
example = dataset["train"][0]

example

{'pixel_values': tensor([[[ 0.7933, -1.0733, -1.7069,  ..., -1.9467, -1.9467, -1.9467],
          [ 0.8104, -1.3473, -1.9467,  ..., -2.1179, -2.1179, -2.1179],
          [ 0.8104, -1.3815, -1.9809,  ..., -2.1179, -2.1179, -2.1179],
          ...,
          [ 0.8276, -1.7240, -2.1179,  ..., -1.9467, -1.9295, -1.9467],
          [ 0.8276, -1.6042, -2.1008,  ..., -1.9980, -1.9980, -2.0665],
          [ 0.8276, -1.6042, -2.0837,  ..., -2.1179, -2.0152, -1.8782]],
 
         [[ 0.9230, -0.9328, -1.5455,  ..., -1.6506, -1.6506, -1.6506],
          [ 0.9230, -1.2654, -1.8431,  ..., -2.0357, -2.0357, -2.0357],
          [ 0.9055, -1.2829, -1.8782,  ..., -2.0357, -2.0357, -2.0357],
          ...,
          [ 0.9755, -1.6331, -2.0357,  ..., -1.7381, -1.7381, -1.7556],
          [ 0.9755, -1.5105, -1.9832,  ..., -1.8081, -1.8081, -1.8782],
          [ 0.9580, -1.4930, -1.9832,  ..., -1.9657, -1.8431, -1.7031]],
 
         [[ 1.0539, -0.5844, -1.0201,  ..., -0.9330, -0.9330, -0.9330],
          [ 

In [13]:
def collate_fn(batch):
    pixel_values = [item["pixel_values"] for item in batch]
    encoding = teacher_processor.pad(pixel_values, return_tensors="pt")
    labels = [item["labels"] for item in batch]
    batch = {}
    batch["pixel_values"] = encoding["pixel_values"]
    batch["labels"] = labels
    return batch

In [14]:
device = "cuda:0" if torch.cuda.is_available() else "cpu" 

teacher_model = teacher_model.to(device)

In [15]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="finetuned-yolos-tiny",
    per_device_train_batch_size=8,
    num_train_epochs=10,
    fp16=True,
    save_steps=200,
    logging_steps=50,
    learning_rate=1e-5,
    weight_decay=1e-4,
    save_total_limit=2,
    remove_unused_columns=False,
)

In [16]:
from transformers import Trainer

trainer = Trainer(
    model=teacher_model,
    args=training_args,
    data_collator=collate_fn,
    train_dataset=dataset["train"], # type: ignore
    tokenizer=teacher_processor,
)

trainer.train()

c:\source\repos\Models-Inferencing-On-Client-Sandbox\sandboxenv\Lib\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  6%|▌         | 50/810 [00:15<03:43,  3.40it/s]

{'loss': 2063.6431, 'grad_norm': 42.59532165527344, 'learning_rate': 9.419753086419753e-06, 'epoch': 0.62}


 12%|█▏        | 100/810 [00:30<03:30,  3.37it/s]

{'loss': 1899.5256, 'grad_norm': 39.595062255859375, 'learning_rate': 8.814814814814817e-06, 'epoch': 1.23}


 19%|█▊        | 150/810 [00:45<03:11,  3.46it/s]

...

 99%|█████████▉| 800/810 [04:04<00:03,  3.27it/s]

{'loss': 1989.5583, 'grad_norm': 11.371615409851074, 'learning_rate': 1.728395061728395e-07, 'epoch': 9.88}


100%|██████████| 810/810 [04:07<00:00,  3.27it/s]

{'train_runtime': 247.8369, 'train_samples_per_second': 25.985, 'train_steps_per_second': 3.268, 'train_loss': 1970.4979021990741, 'epoch': 10.0}


TrainOutput(global_step=810, training_loss=1970.4979021990741, metrics={'train_runtime': 247.8369, 'train_samples_per_second': 25.985, 'train_steps_per_second': 3.268, 'train_loss': 1970.4979021990741, 'epoch': 10.0})

In [17]:
tuned_model_path = "tuned_yolostiny_model"

teacher_model.save_pretrained(tuned_model_path)
teacher_processor.save_pretrained(tuned_model_path)

['tuned_yolostiny_model\\preprocessor_config.json']